In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import random
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Output, Input, State
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn import svm
from sklearn.metrics import root_mean_squared_error
from sklearn.neighbors import KNeighborsRegressor


df_original = sns.load_dataset("iris")

df = df_original.copy()

# codificando las variables nominales para el algoritmo

df["species_encoded"] = LabelEncoder().fit_transform(df["species"])

pca_model = PCA(n_components=2) 

pca = pca_model.fit_transform(df[df.columns[:4]])

df["PCA_1"] = pca.T[0]
df["PCA_2"] = pca.T[1]

df

#### Feature Engireering
Es el proceso de seleccionar, eliminar y transformar las variables de un conjunto para generar una entrada de datos eficaz en un modelo específico, una popular técnica es el Análisis de Componentes Principales(PCA), cuyo fundamento se basa en conceptos de álgebra lineal y su objetivo en la reducción de la dimensionalidad pero conservando a su vez la mayor cantidad de información, proyectando una especie de sombra en los datos. Se implementa a la hora de combatir la maldición de la dimensionalidad, permitir crear gráficos cartesianos y estimular a algoritmos que no sean robustos, como en este ejemplo.

In [ ]:
SVM = svm.SVC(kernel="rbf", C=1, probability=True)

SVM.fit(pca, df["species_encoded"])

x_min, x_max = df["PCA_1"].min() - 1, df["PCA_1"].max() + 1
y_min, y_max = df["PCA_2"].min() - 1, df["PCA_2"].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.1), np.arange(y_min, y_max, 0.1))

Z = SVM.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

plt.contourf(xx, yy, Z, alpha=0.8)
plt.scatter(df["PCA_1"], df["PCA_2"], c=df["species_encoded"], s=20, edgecolor='k')
plt.xlabel('Característica 1')
plt.ylabel('Característica 2')
plt.title('Ajuste de modelo SVM con kernel radial básico y C=1')
plt.show()

#### Máquina de soporte vectorial
Dado que se redujo la dimensión de características se hace más claro definir los conceptos de este algoritmo de clasificaión. Al momento de recibir los datos se define uno o más umbrales que se separan una categoría de la otra, a este umbral se lo denomina hiperplano y en este caso se traza en una distribución de puntos en 2 dimensiones, siendo también aplicado en dimensiones más grandes. Su principal elemento es el hiperplano, que es la frontera que divide todas las observaciones funcionando como una grieta entre las categorías, esto se obtiene mediante la separación a la mitad sobre la distancia entre los puntos de datos más cercanos entre una clase y otra(vectores de soporte), el objetivo final de su ecuación es generar el hiperplano óptimo, es decir, la línea divisora con el mayor margen(distancia entre los vectores de soporte y el hiperplano) posible. 

In [ ]:
var = df.loc[(df["species"] == "virginica") | (df["species"] == "setosa"), ["PCA_1","PCA_2"]]
var["PCA_1"] = round(var["PCA_1"]*100).astype(int)
var["PCA_2"] = round(var["PCA_2"]*100).astype(int)

def add_values(length, value_min, value_max):
    new_values = np.array([])
    for i in range(round(length)):
        value = random.randint(value_min, value_max)
        new_values = np.append(new_values, value)
    return new_values
    
grp1_pca_1 = add_values(len(var)/2, 100, 400)
grp1_pca_2 = add_values(len(var)/2, -100, 100)
grp2_pca_1 = add_values(len(var)*1.5, -400, -200)
grp2_pca_2 = add_values(len(var)*1.5, -100, 100)

new_data_1 = np.array([grp1_pca_1,grp1_pca_2]).T
new_data_2 = np.array([grp2_pca_1,grp2_pca_2]).T

df_new_data_1 = pd.DataFrame(new_data_1, columns=["PCA_1","PCA_2"])
df_new_data_2 = pd.DataFrame(new_data_2, columns=["PCA_1","PCA_2"])

var = pd.concat([var, df_new_data_1, df_new_data_2])


fig, ax = plt.subplots(1,3, figsize=(18,7))

def get_function_objetive(i, data_split):
    var["data_split"] = data_split
    knn_regressor = KNeighborsRegressor(n_neighbors=150)
    knn_regressor.fit(var.loc[var["data_split"] == "data_train", "PCA_2"].values.reshape(-1,1), var.loc[var["data_split"] == "data_train", "PCA_1"])
    RMSE = root_mean_squared_error(knn_regressor.predict(var.loc[var["data_split"] == "data_test", "PCA_2"].values.reshape(-1,1)), var.loc[var["data_split"] == "data_test", "PCA_1"])
    sns.scatterplot(var, x="PCA_2", y="PCA_1", hue="data_split", ax=ax[i])
    ax[i].grid("on")
    ax[i].set_title(f"RMSE: {str(RMSE)[:5]}")

get_function_objetive(0, np.where(var["PCA_1"] < -195, "data_train", "data_test"))
get_function_objetive(1, np.where((var["PCA_1"] <= -200) & (var["PCA_2"] < 40) | (var["PCA_1"] > 0) & (var["PCA_2"] > 0), "data_train", "data_test"))
get_function_objetive(2, np.where((var["PCA_1"].between(-300,-200)) | (var["PCA_1"].between(200,400)), "data_train", "data_test"))

plt.show()

#### Sobreajuste
En la anterior celda se extrajeron específicamente un par de variables continuas formando un subconjunto sin contexto y que no brinda ninguna información en particular, esto se hizo para explicar un concepto recurrente en el aprendizaje automático y que está fuertemente relacionado con otra definición vista anteriormente(proyecto 2), siendo esta el error de varianza. El sobreajuste consiste en un modelo que se que se adaptó de forma elevada un grupo de datos, dejándolo poco preparado para afrontar nuevas observaciones. En la anterior figura se muestran las diferentes selecciones de datos de entrenamiento y prueba de cada modelo junto con su función objetivo que varía según las muestras. En este caso, se encuentra un umbral que diferencia dos agrupaciones con el fin de ilustrar un modelo conservador a sus objetos y como el cambio de elección de datos provoca un nuevo desempeño de este.


In [ ]:
object = [[df["sepal_length"].mean(), df["sepal_width"].mean(), df["petal_length"].mean(), df["petal_width"].mean()]]

pca_object = pca_model.transform(object)

# prediciendo la clase del nuevo objeto

predict_encoded = SVM.predict(pca_object)

# asociando las variables codificadas con sus versiones originales

classes = list(zip(df["species"].unique(),df["species_encoded"].unique()))

# asociando la predicción a su clase

predict_example = classes[predict_encoded[0]][0]

probability = SVM.predict_proba(pca_object)

probability = probability[0, predict_encoded]*100

probability = str(probability[0])

print("--------------------------------------------")
print("Nuevo objeto \n")
for c in df.columns[:4]:
    print(f"{c}: {df[c].mean()}")
    
print(f"\npredicción: {predict_example} | probabilidad: {probability[:4]}%")
print("--------------------------------------------")

#### Dashboard interactivo que refleja la transformación de varibles y permite introducir datos para clasificarlos


In [ ]:
app = dash.Dash(__name__)

setosa = df.loc[df["species"] == "setosa",:]
virginica = df.loc[df["species"] == "virginica",:]
versicolor = df.loc[df["species"] == "versicolor",:]

graph_pca = go.Figure()
graph_pca.add_trace(go.Scatter(x=df["PCA_1"],y=df["PCA_2"],mode="markers",marker_color="blue",name="especies"))
graph_pca.update_layout(title="Figura PCA(principal components analysis)")
graph_pca.update_layout(legend=dict(font=dict(size=9)))

predict_text = html.B(children=[],id="predict",style={})
probability_text =  html.B(children=[],id="probability")

colors_species = {
    "setosa":"blue",
    "virginica":"green",
    "versicolor":"red"
}

app.layout =  html.Div(id="body",className="e4_body",children=[
    html.H1("Clasificador de especies",id="title",className="e4_title"),
    html.Div(id="dashboard",className="e4_dashboard",children=[
        html.Div(className="e4_info_div",children=[
           html.H2("Iris medias",id="title_2",className="e4_title_2"),
           html.Div(className="e4_info",children=[
              html.Img(id="img",className="e4_img", src="assets/iris.png"),
              html.Ul(className="e4_ul_div",children=[
              html.Ul(id="ul",className="e4_ul", children=[
                html.Li(f"longitud sétalo: {round(setosa["sepal_length"].mean())}"),
                html.Li(f"ancho sétalo: {round(setosa["sepal_width"].mean())}"),
                html.Li(f"longitud pétalo: {round(setosa["petal_length"].mean())}"),
                html.Li(f"ancho pétalo: {round(setosa["petal_width"].mean())}")
              ]),
              html.Ul(className="e4_ul", children=[
                html.Li(f"longitud sétalo: {round(versicolor["sepal_length"].mean())}"),
                html.Li(f"ancho sétalo: {round(versicolor["sepal_width"].mean())}"),
                html.Li(f"longitud pétalo: {round(versicolor["petal_length"].mean())}"),
                html.Li(f"ancho pétalo: {round(versicolor["petal_width"].mean())}")
              ]),
              html.Ul(className="e4_ul", children=[
                html.Li(f"longitud sétalo: {round(virginica["sepal_length"].mean())}"),
                html.Li(f"ancho sétalo: {round(virginica["sepal_width"].mean())}"),
                html.Li(f"longitud pétalo: {round(virginica["petal_length"].mean())}"),
                html.Li(f"ancho pétalo: {round(virginica["petal_width"].mean())}")
              ])
            ])
           ])
        ]),
        html.Div(className="e4_graph_div",children=[
            dcc.Graph(id="graph-2",className="e4_graph",figure=graph_pca),
            html.Form(id="input_div",className="input_div",children=[
                dcc.Input(id="input_1",className="input",type="text",placeholder="longitud_sépalo",size="7"),
                dcc.Input(id="input_2",className="input",type="text",placeholder="ancho_sépalo",size="7"),
                dcc.Input(id="input_3",className="input",type="text",placeholder="longitud_pétalo",size="7"),
                dcc.Input(id="input_4",className="input",type="text",placeholder="ancho_pétalo",size="7"),
                html.Button("enviar",id="button",type="button",className="input",n_clicks=0)
            ]),
            html.P(["predicción: ",predict_text," | probabildad: ",probability_text,"%"],className="e4_predict")
        ])
    ])
])
        
@app.callback(
    [Output(component_id="graph-2",component_property="figure"),
    Output(component_id="predict",component_property="children"),
    Output(component_id="probability",component_property="children"),
    Output(component_id="predict",component_property="style"),
    Output(component_id="probability",component_property="style")],
    [Input(component_id="button",component_property="n_clicks")],
    [State(component_id="input_1",component_property="value"),
    State(component_id="input_2",component_property="value"),
    State(component_id="input_3",component_property="value"),
    State(component_id="input_4",component_property="value")]
)

def update_graph(n_clicks, var_1, var_2, var_3, var_4):
    if n_clicks is not None and n_clicks > 0:
        object = [[var_1,var_2,var_3,var_4]]    
        pca_object = pca_model.transform(object)
        predict_encoded = SVM.predict(pca_object)
        predict = classes[predict_encoded[0]][0]
        predict_color = {"color":colors_species[predict]}
        probability_color = {"color":colors_species[predict]}
        probability = SVM.predict_proba(pca_object)
        probability = probability[0,predict_encoded] * 100
        probability = str(probability[0])
        probability = probability[:4]
        graph_pca.add_trace(go.Scatter(x=[pca_object[0,0]], y=[pca_object[0,1]], mode="markers", marker_color="red", name=f"nueva especie({predict})"))
    
    return graph_pca, predict, probability, predict_color, probability_color


if __name__ == "__main__":
    app.run_server(debug=False)

#### Machine Learning Deployment
Hasta ahora hemos generado modelos predictivos que se encargan de resolver problemas asignados pero no fueron llevados a producción, es decir, tomar el modelo en particular desarrollado a partir de los fundamentos de Machine Learning, ponerlo a disposición de un usuario final y lograr que este modelo se pueda actualizar periódicamente para garantizar que tenga siempre el desempeño esperado. Para esto se requieren de habilidades menos científicas y más propiamente de un Ingenirero de software, donde se implementan herramientas como Docker que ayuda a crear y distribuir aplicaciones mediante contenedores que incluyan las librerías específicas instaladas, habiliten puertos, contengan volumes, etc y luego del uso de un orquestador como Kubernetes donde se gestionarán los recursos en los contenedores, servicios, configuraciones y la infraestructura en general del despliegue que se realizará. Como ya habíamos mencionado, esta área es un híbrido entre la ciencia de datos que abarca procesos como la extracción de datos, el análisis exploratorio, el preprocesamiento de datos, el entrenamiento de modelos y su evaluación y un ingeniero de softwre convencional, sin embargo, en el Machine Learning Deployment el software es mayormente dinámico, ya que el constante cambio en la información provoca que se requiera de fases posteirores al despliegue y puesta en servicio denominadas monitoreo y mantenimiento, el objetivo en estas fases es la verificación del desempeño del modelo y la repetición de las fases anteriormente mencionadas en el caso de que haya habido una degradación.